# QAOA
VQEアルゴリズムとほぼ同じですが、効率的に解を探索する量子回路に組合せ最適化問題向けのを利用します。

## 今回学ぶこと
1. QAOAの仕組み
2. 簡単な例題でQAOAの実装をする

## Blueqatのインストール
pipからBlueqatをインストールします。

In [2]:
!pip install -U blueqat

Requirement already up-to-date: blueqat in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (0.3.14)
Requirement not upgraded as not directly required: scipy>=1.1.0 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from blueqat) (1.1.0)
Requirement not upgraded as not directly required: numpy~=1.12 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from blueqat) (1.14.6)
numba 0.49.0 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
You are using pip version 10.0.1, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


----

## 固有値問題
QAOAは問題設定された行列であるハミルトニアンを準備し、固有値問題を解きます。解きたい問題をコストハミルトニアン$H_{cost}$として、対応する固有値を$E_0$として、

$$
H_{cost}\mid \psi \rangle = E_0\mid \psi \rangle
$$

となる$E_0$と$\mid \psi \rangle$を探すのが目的です。

## 量子断熱計算
固有値は量子断熱計算を使って計算します。現時刻をt、全体スケジュールをTとし、ハミルトニアンは下記の二つを準備します。

1. 問題の制約条件から設定する、ミキサーハミルトニアン$H_{mixer}$  
2. 問題の最小化したい、コストハミルトニアン $H_{cost}$ 

ミキサーハミルトニアンからスタートし、コストハミルトニアンへと時間をかけて入れ替えを行います。

$$
H_{temp} = (1-\frac{t}{T})H_{mixer} + \frac{t}{T}H_{cost}
$$

ミキサーハミルトニアンに簡単な問題を設定し、コストハミルトニアンに難しい問題を設定することで、答えの状態を保ったまま入れ替えを行うことができます。

$T\rightarrow\infty$ のとき、途中過程は以下のような固有状態を保ち続けます。

$$
H_{temp}\mid \psi \rangle = E_{0temp}\mid \psi \rangle
$$

下記の時間発展演算子を少しずつ量子ゲートとしてかけることで二つのハミルトニアンを入れ替える過程を行えます。

$$
\mid \psi_{t+1} \rangle = e^{-iHt}  \mid \psi_t \rangle
$$

QAOA Ansatzの中にはミキサーハミルトニアンとコストハミルトニアンが時間発展演算子に変形され格納されています。$X$ゲートの時間発展は$RX(\theta)$ゲートのような任意回転ゲートを利用します。また、2量子ビットは$XX$ゲートの時間発展も、$RXX(\theta)$のように任意回転のイジングカップリングゲートが利用されます。

また、量子断熱計算は通常アナログの過程を時間発展演算子を利用して離散化しています。より精度を上げるためには、分割数を増やすことで精度を上げられますが、回路が長くなるので、その分計算時間がかかります。QAOAでもステップ数を設定することで精度を上げることができます。ステップ数はQAOA Ansatzの中の繰り返しの回数に対応しています。

## 量子回路
量子回路は、

1. 初期状態準備
2. QAOA Ansatzの実行
3. 測定

で構成されます。具体的な回路例を使って下記のようになります。

```
|0> --H---------------RZZ--RZ--RX-----RZZ--RZ--RX------[測定]
|0> --H---------------*----RZ--RX-----*----RZ--RX------[測定]
      [初期状態準備]           [QAOA Ansatz]
```

一番左が初期状態の準備、次がQAOA Ansatzです。QAOA Ansatzは似たステップが繰り返されています。最後に測定をして解を求めます。

## ミキサーハミルトニアンと初期状態の準備
ミキサーハミルトニアンは問題の探索の仕方を決めるためのものです。問題の制約条件によって決めると、初期状態が決まります。「初期状態の準備」は、ミキサーハミルトニアンの固有状態を設定します。下記に代表的なミキサーハミルトニアンと対応する初期状態をまとめました。

## Xミキサー
代表的な量子ビットの値を反転させるXミキサーです。量子アニーリングで利用されます。

$$
X\mid 0 \rangle = 
\begin{bmatrix}
0&1\\
1&0
\end{bmatrix}
\begin{bmatrix}
1\\
0
\end{bmatrix}
=
\begin{bmatrix}
0\\
1
\end{bmatrix}
$$

対応する初期状態は、

$$
\mid + \rangle$$

となり、通常はHゲートを量子ビットに適用します。QAOAのデフォルトのミキサーです。

## XYミキサー
XYミキサーは二つの状態を入れ替えながら探索を行います。(XX+YY)/2は、

$$
X_0X_1 + Y_0Y_1 = 
\begin{bmatrix}
0&1\\
1&0
\end{bmatrix}
\otimes
\begin{bmatrix}
0&1\\
1&0
\end{bmatrix}
+
\begin{bmatrix}
0&-i\\
i&0
\end{bmatrix}
\otimes
\begin{bmatrix}
0&-i\\
i&0
\end{bmatrix}
=
\begin{bmatrix}
0&0&0&1\\
0&0&1&0\\
0&1&0&0\\
1&0&0&0
\end{bmatrix}
+
\begin{bmatrix}
0&0&0&-1\\
0&0&1&0\\
0&1&0&0\\
-1&0&0&0
\end{bmatrix}
$$

ですので、足し合わせて2で割ると、

$$
(X_0X_1 + Y_0Y_1)/2 
=
\begin{bmatrix}
0&0&0&0\\
0&0&1&0\\
0&1&0&0\\
0&0&0&0
\end{bmatrix}
$$

となりました。こちらは、01と10状態を交換しながら探索を行います。初期状態は固有状態として、01と10のもつれ状態

$$
|01> + |10>
$$

などを選びます。また、

$$
(X_0X_1 - Y_0Y_1)/2 = 
\begin{bmatrix}
0&0&0&1\\
0&0&0&0\\
0&0&0&0\\
1&0&0&0
\end{bmatrix}
$$

となり、こちらは00と11の状態を交換します。初期状態は00と11のもつれ状態

$$
|00> + |11>
$$

などを選びます。

## コストハミルトニアンと最終状態の取得
コストハミルトニアンは問題によって最小化したいものを条件として設定します。社会問題の組合せ最適化問題を、物理のイジングモデルの上に落とし込みます。

$$
H_{cost} = -\sum h_i Z_i -\sum J_{ij} Z_i Z_j
$$

コストハミルトニアンはZ演算子の期待値が+1と-1であることを利用して、問題の条件をZもしくはZZ演算子を使って表現します。

```
h = -Z(0) - Z(0)*Z(1)
```

単体のZの前に「バイアス」を設定し、複数のZの前に「ウェイト」を設定します。

## 例題：量子アニーリング

QaoaAnsatzのデフォルト初期状態は量子アニーリングに設定されています。ミキサーハミルトニアンにXミキサーを使い、初期状態に|+>を設定しています。初期状態の|+>は全ての量子ビットにアダマールゲートHをかけることで実現します。

```
|0> --H--  --RZZ--RZ--RX- -RZZ--RZ--RX--  --[測定]
|0> --H--  --*--------RX- -*--------RX--  --[測定]
```

In [4]:
import numpy as np
from blueqat import Circuit
from blueqat.pauli import X, Y, Z, I
from blueqat.pauli import qubo_bit as q
from blueqat import vqe

#mixerの準備
Xmixer = X[0]+X[1]
Xinit = Circuit(2).h[0,1]

h = -Z(0) -Z(0)*Z(1)
step = 2

result = vqe.Vqe(vqe.QaoaAnsatz(h, step, Xinit, Xmixer)).run()
print(result.most_common(12))

(((0, 0), 0.8924275962214443), ((1, 1), 0.06107032069693087), ((0, 1), 0.045131398638712225), ((1, 0), 0.0013706844429107316))


初期状態とミキサーを指定しない場合は量子アニーリングが採用されます。

In [5]:
result = vqe.Vqe(vqe.QaoaAnsatz(h, step)).run()
print(result.most_common(12))

(((0, 0), 0.9956500173694675), ((0, 1), 0.0042136058253980325), ((1, 1), 9.452266153620642e-05), ((1, 0), 4.1854143597953274e-05))


## 例題：XYミキサー
同じ問題でXYミキサーを使います。初期状態には|01>と|10>のもつれ状態を指定します。ミキサーハミルトニアンには(XX+YY)/2のハミルトニアンを渡します。

In [6]:
#mixerの準備
XYmixer = 0.5*X[0]*X[1] + 0.5*Y[0]*Y[1]
XYinit = Circuit().h[0].cx[0,1].x[0]

result = vqe.Vqe(vqe.QaoaAnsatz(h, step, XYinit, XYmixer)).run()
print(result.most_common(12))

(((0, 1), 0.8999090399482286), ((1, 0), 0.1000909600517694), ((0, 0), 6.162975822039155e-33), ((1, 1), 5.525199607057015e-33))


となりました。本来は(0,0)が出て欲しいところですが、01と10で制約をかけましたので、01と10の答えの中から正解に近いものを探そうとしています。